In [1]:
#beautiful soup imports
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import time
import random
from time import sleep

# OnTheSnow Webscraping

Importing a list of URLS that has been pulled from [github](https://github.com/SijiaLai/OnTheSnow/tree/master) and later updated in excel to include additional resorts.

In [2]:
#importing
url_df = pd.read_csv('data/on_the_snow_url.csv')

In [3]:
url_df

,url
0,https://www.onthesnow.com/california/squaw-val...
1,https://www.onthesnow.com/california/mammoth-m...
2,https://www.onthesnow.com/california/donner-sk...
3,https://www.onthesnow.com/california/sugar-bow...
4,https://www.onthesnow.com/california/kirkwood/...
...,...
332,https://www.onthesnow.com/oregon/mt-bachelor/s...
333,https://www.onthesnow.com/michigan/mount-bohem...
334,https://www.onthesnow.com/pennsylvania/shawnee...
335,https://www.onthesnow.com/massachusetts/wachus...


In [4]:
#saving urls as a list
url_list = url_df['url'].tolist()

To begin, I made a function that parses through the list of links and checks to see if they're still active. If they were not active, I manually updated the links in the url csv.

In [5]:
#making a list of all urls and removing .html to test links
test_df = url_df.replace(".html", "", regex=True)
test_list = test_df['url'].tolist()

In [6]:
#making a function to check if all links are active
def check_link_status(link):
    try:
        response = requests.head(link, timeout=5)
        return response.status_code == 200
    except requests.exceptions.RequestException:
        return False

In [7]:
#checking to see if all links are active
for link in test_list:
    if check_link_status(link):
        print(f"{link} is active.")
    else:
        print(f"{link} is not active or cannot be reached.")

https://www.onthesnow.com/california/squaw-valley-usa/ski-resort is active.
https://www.onthesnow.com/california/mammoth-mountain-ski-area/ski-resort is active.
https://www.onthesnow.com/california/donner-ski-ranch/ski-resort is active.
https://www.onthesnow.com/california/sugar-bowl-resort/ski-resort is active.
https://www.onthesnow.com/california/kirkwood/ski-resort is active.
https://www.onthesnow.com/california/boreal/ski-resort is active.
https://www.onthesnow.com/california/sierra-at-tahoe/ski-resort is active.
https://www.onthesnow.com/nevada/mt-rose-ski-tahoe/ski-resort is active.
https://www.onthesnow.com/california/soda-springs/ski-resort is active.
https://www.onthesnow.com/colorado/wolf-creek-ski-area/ski-resort is active.
https://www.onthesnow.com/washington/49-degrees-north/ski-resort is active.
https://www.onthesnow.com/california/bear-valley/ski-resort is active.
https://www.onthesnow.com/utah/alta-ski-area/ski-resort is active.
https://www.onthesnow.com/oregon/mt-hood-

https://www.onthesnow.com/montana/maverick-mountain/ski-resort is active.
https://www.onthesnow.com/montana/showdown-ski-area/ski-resort is active.
https://www.onthesnow.com/new-york/song-mountain/ski-resort is active.
https://www.onthesnow.com/vermont/stowe-mountain-resort/ski-resort is active.
https://www.onthesnow.com/pennsylvania/big-boulder/ski-resort is active.
https://www.onthesnow.com/minnesota/buck-hill/ski-resort is active.
https://www.onthesnow.com/michigan/crystal-mountain/ski-resort is active.
https://www.onthesnow.com/vermont/killington-resort/ski-resort is active.
https://www.onthesnow.com/michigan/shanty-creek/ski-resort is active.
https://www.onthesnow.com/vermont/smugglers-notch-resort/ski-resort is active.
https://www.onthesnow.com/new-hampshire/wildcat-mountain/ski-resort is active.
https://www.onthesnow.com/new-mexico/sipapu-ski-and-summer-resort/ski-resort is active.
https://www.onthesnow.com/new-hampshire/bretton-woods/ski-resort is active.
https://www.onthesnow.

https://www.onthesnow.com/pennsylvania/eagle-rock/ski-resort is active.
https://www.onthesnow.com/alaska/eaglecrest-ski-area/ski-resort is active.
https://www.onthesnow.com/pennsylvania/elk-mountain-ski-resort/ski-resort is active.
https://www.onthesnow.com/wisconsin/grand-geneva/ski-resort is active.
https://www.onthesnow.com/new-york/greek-peak/ski-resort is active.
https://www.onthesnow.com/missouri/hidden-valley-ski-area/ski-resort is active.
https://www.onthesnow.com/colorado/howelsen-hill/ski-resort is active.
https://www.onthesnow.com/pennsylvania/jack-frost/ski-resort is active.
https://www.onthesnow.com/idaho/kelly-canyon-ski-area/ski-resort is active.
https://www.onthesnow.com/new-hampshire/king-pine/ski-resort is active.
https://www.onthesnow.com/new-york/kissing-bridge/ski-resort is active.
https://www.onthesnow.com/new-york/labrador-mt/ski-resort is active.
https://www.onthesnow.com/maine/lost-valley/ski-resort is active.
https://www.onthesnow.com/ohio/mad-river-mountain/s

https://www.onthesnow.com/oregon/mt-bachelor/ski-resort is active.
https://www.onthesnow.com/michigan/mount-bohemia/ski-resort is active.
https://www.onthesnow.com/pennsylvania/shawnee-mountain-ski-area/ski-resort is active.
https://www.onthesnow.com/massachusetts/wachusett-mountain-ski-area/ski-resort is active.
https://www.onthesnow.com/vermont/magic-mountain/ski-resort is active.


This code was also adapted from the same github user. I updated the code as not all of the html elements were still the same, and included additional features that I wanted to scrape.

In [294]:
def is_good_response(resp):
    """
    Ensures that the response is a html.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 and
            content_type is not None
            and content_type.find('html') > -1)

def get_html_content(url):
    """
    Get the contents of the url.
    """
    time.sleep(1)
    try:
        with closing(get(url, stream=True)) as resp:
            content_type = resp.headers['Content-Type'].lower()
            if is_good_response(resp):
                return resp.content
            else:
                raise RequestException('Error: Received status code {0} for URL {1}'.format(resp.status_code, url))
    except RequestException as e:
        print('Error during requests to {0} : {1}'.format(url, str(e)))
        raise

def get_basic_resort_statistics(resortUrl):
    """
    Get basic stat for the ski resort.
    """
    resortContent = get_html_content(resortUrl)
    resortHtml = BeautifulSoup(resortContent, 'html.parser')

    stat = {}

    #Name
    if (resortHtml.find("span",{"class":"styles_resort__1wghR"}) != None):
        resortNameContent = resortHtml.find("span",{"class":"styles_resort__1wghR"})
        resortName = resortNameContent.get_text()
        stat["ski_resort"] = resortName
    else:
        nameNote = "Check name info."
        stat["name_note"] = nameNote

    # Locations
    if resortHtml.find("div", {"class": "styles_address__1Mkk8"}) is not None:
        resortLocationContent = resortHtml.find("div", {"class": "styles_address__1Mkk8"})
        address_elements = resortLocationContent.find_all("p")
        if len(address_elements) >= 3:
            address = address_elements[0].get_text()
            city_state = address_elements[1].get_text().split(",")
            city = city_state[0].strip()
            state = city_state[1].strip()
            country = address_elements[2].get_text()
            stat["address"] = address
            stat["city"] = city
            stat["state"] = state
            stat["country"] = country
        
        else:
            locationNote = "Incomplete location info."
            stat["location_note"] = locationNote

    # Elevation
    if resortHtml.find("div", {"class": "styles_elevation__2L-EV"}) is not None:
        elevationContent = resortHtml.find("div", {"class": "styles_elevation__2L-EV"})
        elevationValues = elevationContent.find_all("div", {"class": "styles_inner__2bSAI"})
        keys = ["summit", "drop", "base"]
        for key, value in zip(keys, elevationValues):
            distance = value.find("div", {"class": "styles_distance__3eFw3"}).get_text().replace("'", "")
            distance = int(distance) if distance.isdigit() else None
            stat[key] = distance
    else:
        elevationNote = "Check elevation information."
        stat["elevation_note"] = elevationNote

    # Gondolas and Lifts
    if resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Gondolas & Trams"}) is not None:
        # Extract the number of trams
        trams_div = resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Gondolas & Trams"})
        trams_value = trams_div.find("div", {"class": "styles_value__fB0LV"}).text
        trams = float(trams_value)
        stat["gondolas_and_trams"] = trams

    if resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "High Speed Quads"}) is not None:
        # Extract the number of fast eight lifts
        fast_eight_div = resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "High Speed Quads"})
        fast_eight_value = fast_eight_div.find("div", {"class": "styles_value__fB0LV"}).text
        fast_eight = float(fast_eight_value)
        stat["fast_eight"] = fast_eight
        
    if resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "High Speed Sixes"}) is not None:
        # Extract the number of high-speed sixes lifts
        high_speed_sixes_div = resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "High Speed Sixes"})
        high_speed_sixes_value = high_speed_sixes_div.find("div", {"class": "styles_value__fB0LV"}).text
        high_speed_sixes = float(high_speed_sixes_value)
        stat["high_speed_sixes"] = high_speed_sixes

    if resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Quad Chairs"}) is not None:
        # Extract the number of quad chairs lifts
        quad_chairs_div = resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Quad Chairs"})
        quad_chairs_value = quad_chairs_div.find("div", {"class": "styles_value__fB0LV"}).text
        quad_chairs = float(quad_chairs_value)
        stat["quad_chairs"] = quad_chairs

    if resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Triple Chairs"}) is not None:
        # Extract the number of triple chairs lifts
        triple_chairs_div = resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Triple Chairs"})
        triple_chairs_value = triple_chairs_div.find("div", {"class": "styles_value__fB0LV"}).text
        triple_chairs = float(triple_chairs_value)
        stat["triple_chairs"] = triple_chairs
        
    if resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Double Chairs"}) is not None:
        # Extract the number of double_chairs lifts
        double_chairs_div = resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Double Chairs"})
        double_chairs_value = double_chairs_div.find("div", {"class": "styles_value__fB0LV"}).text
        double_chairs = float(double_chairs_value)
        stat["double_chairs"] = double_chairs 
        
    if resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Surface Lifts"}) is not None:
        # Extract the number of triple chairs lifts
        surface_lifts_div = resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Surface Lifts"})
        surface_lifts_value = surface_lifts_div.find("div", {"class": "styles_value__fB0LV"}).text
        surface_lifts = float(surface_lifts_value)
        stat["surface_lifts"] = surface_lifts  
        
    else:
        gondolas_lifts_note = "Check gondola information."
        stat["gondolas_lifts_note"] = gondolas_lifts_note

    #Terrain
    if resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Runs in Total"}) is not None:
        # Extract the number of runs
        run_div = resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Runs in Total"})
        run_value = run_div.find("div", {"class": "styles_value__fB0LV"}).text
        run = float(run_value)
        stat["total_runs"] = run
        
    if resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Longest Run"}) is not None:
        # Extract the longest run
        longest_run_div = resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Longest Run"})
        longest_run_value = longest_run_div.find("div", {"class": "styles_value__fB0LV"}).text
        longest_run = str(longest_run_value)
        stat["longest_run"] = longest_run
        
    if resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Skiable Terrain"}) is not None:
        # Extract the skiable terrain
        skiable_div = resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Skiable Terrain"})
        skiable_value = skiable_div.find("div", {"class": "styles_value__fB0LV"}).text
        skiable = str(skiable_value)
        stat["skiable_terrain"] = skiable
        
    if resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Snow Making"}) is not None:
        # Extract the snow making
        snowmak_div = resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Snow Making"})
        snowmak_value = snowmak_div.find("div", {"class": "styles_value__fB0LV"}).text
        snowmak = str(snowmak_value)
        stat["snow_making"] = snowmak
        
    if resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Night Skiing"}) is not None:
        # Extract the snow making
        night_ski_div = resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Night Skiing"})
        night_ski_value = night_ski_div.find("div", {"class": "styles_value__fB0LV"}).text
        night_ski = str(night_ski_value)
        stat["night_skiing"] = night_ski
        
    if resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Beginners Runs"}) is not None:
        # Extract the snow making
        begrun_div = resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Beginners Runs"})
        begrun_value = begrun_div.find("div", {"class": "styles_value__fB0LV"}).text
        begrun = str(begrun_value)
        stat["beginner_runs"] = begrun
        
    if resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Beginners Runs"}) is not None:
        # Extract the snow making
        begrun_div = resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Beginners Runs"})
        begrun_value = begrun_div.find("div", {"class": "styles_value__fB0LV"}).text
        begrun = str(begrun_value)
        stat["beginner_runs"] = begrun
        
    if resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Intermediate Runs"}) is not None:
        # Extract the snow making
        intrun_div = resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Intermediate Runs"})
        intrun_value = intrun_div.find("div", {"class": "styles_value__fB0LV"}).text
        intrun = str(intrun_value)
        stat["intermediate_runs"] = intrun
        
    if resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Advanced Runs"}) is not None:
        # Extract the snow making
        advrun_div = resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Advanced Runs"})
        advrun_value = advrun_div.find("div", {"class": "styles_value__fB0LV"}).text
        advrun = str(advrun_value)
        stat["advanced_runs"] = advrun

    if resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Expert Runs"}) is not None:
        # Extract the snow making
        exprun_div = resortHtml.find("div", {"class": "styles_box__3XF8j", "title": "Expert Runs"})
        exprun_value = exprun_div.find("div", {"class": "styles_value__fB0LV"}).text
        exprun = str(exprun_value)
        stat["expert_runs"] = exprun      
        
    else:
        terrainNote = "Check terrain information."
        stat["terrain_note"] = terrainNote

    # Days Opened and Snowfall
    if resortHtml.find("div", class_="styles_importantDates__LELVC") is not None:
        important_dates = resortHtml.find("div", class_="styles_importantDates__LELVC")
        rows = important_dates.find_all("div", class_="styles_row__1t2pK")

        for row in rows:
            items = row.find_all("div", class_="styles_item__31bBH")

            for item in items:
                label = item.find("div").get_text(strip=True)
                value = item.find("span", class_="h3 styles_h3__OQalR").get_text(strip=True)

                if label == "Days Open Last Year":
                    daysOpenLastYear = value
                    stat["daysOpenLastYear"] = daysOpenLastYear
                elif label == "Average Snowfall":
                    averageSnowfall = value
                    stat["averageSnowfall"] = averageSnowfall
                    
    else:
        daysopenNote = "Check resort open information."
        stat["open_note"] = daysopenNote
        
        
    # Projected Opening
    if resortHtml.find("div", class_="styles_importantDates__LELVC") is not None:
        important_dates = resortHtml.find("div", class_="styles_importantDates__LELVC")
        rows = important_dates.find_all("div", class_="styles_row__1t2pK")

        for row in rows:
            items = row.find_all("div", class_="styles_item__31bBH")

            for item in items:
                label = item.find("div").get_text(strip=True)
                date_element = item.find("span", class_="styles_desktopDate__1Djk-")

                if date_element is not None:
                    value = date_element.get_text(strip=True)

                    if label == "Projected Opening":
                        projectedOpening = value
                        stat["projectedOpening"] = projectedOpening
                    elif label == "Projected Closing":
                        projectedClosing = value
                        stat["projectedClosing"] = projectedClosing          
    else:
        openNote = "Check resort opening/closing date information."
        stat["open_note"] = openNote

    # Check if  values weren't found
    if "daysOpenLastYear" not in stat:
        stat["days_open_note"] = "Check days info"
    if "projectedOpening" not in stat:
        stat["projected_opening_note"] = "Check projected opening info"
    if "averageSnowfall" not in stat:
        stat["average_snowfall_note"] = "Check average snowfall info"
    if "projectedClosing" not in stat:
        stat["projected_closing_note"] = "Check projected closing info"
        
    # Average Snowfall
    if resortHtml.find("div", class_="styles_box__1sXJN") is not None:
        monthly_snowfall = resortHtml.find("div", class_="styles_box__1sXJN")
        rows = monthly_snowfall.find_all("div", class_="styles_inner__3kHSq")

        for row in rows:
            label = row.find("div", class_="styles_date__ACGKW").get_text(strip=True)
            value = row.find("div", class_="styles_value__3uWEv").get_text(strip=True)

            if label == "Nov":
                stat["nov_snow"] = str(value)
            elif label == "Dec":
                stat["dec_snow"] = str(value)
            elif label == "Jan":
                stat["jan_snow"] = str(value)
            elif label == "Feb":
                stat["feb_snow"] = str(value)
            elif label == "Mar":
                stat["mar_snow"] = str(value)
            elif label == "Apr":
                stat["apr_snow"] = str(value)
            elif label == "May":
                stat["may_snow"] = str(value)

    else:
        averageSnowfallNote = "Check snowfall information."
        stat["snowfall_note"] = averageSnowfallNote
    
    return stat

# def get_resort_prices(resortUrl):
#     """
#     Get the resort lift ticket prices.
#     """
#     prices = {}

#     resortUrl = resortUrl[:-15] + "lift-tickets.html"
#     priceContent = get_html_content(resortUrl)

#     priceHtml = BeautifulSoup(priceContent, 'html.parser')
#     priceData = priceHtml.find_all("span", class_="h4 styles_h4__1nbGO")

#     for i, price in enumerate(priceData):
#         value = str(price.get_text(strip=True))
#         if i < 11:
#             key = ""
#             if i == 0:
#                 key = "child_season"
#             elif i == 1:
#                 key = "junior_season"
#             elif i == 2:
#                 key = "adult_season"
#             elif i == 3:
#                 key = "child_weekday"
#             elif i == 4:
#                 key = "junior_weekday"
#             elif i == 5:
#                 key = "adult_weekday"
#             elif i == 6:
#                 key = "senior_weekday"
#             elif i == 7:
#                 key = "child_weekend"
#             elif i == 8:
#                 key = "junior_weekend"
#             elif i == 9:
#                 key = "adult_weekend"
#             elif i == 10:
#                 key = "senior_weekend"

#             prices[key] = value

#     if len(priceData) < 1:
#         ticketpriceNote = "Check ticket prices."
#         stat["ticket_price_note"] = ticketpriceNote

#     return prices

def get_resort_prices(resortUrl):
    """
    Get the resort lift ticket prices.
    """
    prices = {}

    resortUrl = resortUrl[:-15] + "lift-tickets.html"
    priceContent = get_html_content(resortUrl)

    priceHtml = BeautifulSoup(priceContent, 'html.parser')
    priceData = priceHtml.find('script', id='__NEXT_DATA__')

    if priceData is not None:
        script_text = priceData.string
        ticket_dict = json.loads(script_text)

        daily_list = ['childrenWeekdayPrice', 'childrenWeekendPrice',
            'children2DayPrice', 'children6DayPrice', 'teenagerWeekdayPrice',
            'teenagerWeekendPrice', 'teenagerHalfDayPrice', 'teenager2DayPrice',
            'teenager6DayPrice', 'adultWeekdayPrice', 'adultWeekendPrice',
            'adultHalfDayPrice', 'adult2DayPrice', 'adult6DayPrice',
            'seniorWeekdayPrice', 'seniorWeekendPrice', 'seniorHalfDayPrice',
            'senior2DayPrice', 'senior6DayPrice']

        season_list = ['childrenPrice', 'teenagerPrice',
                        'adultPrice', 'seniorPrice']

        # Loop for daily lift tickets
        for x in daily_list:
            if x in ticket_dict['props']['pageProps']['dailyLiftTickets']:
                prices[x] = ticket_dict['props']['pageProps']['dailyLiftTickets'][x]
            else:
                prices[x] = np.nan

        # Loop for season lift tickets
        for x in season_list:
            if x in ticket_dict['props']['pageProps']['seasonLiftTickets']:
                prices[x + "_season"] = ticket_dict['props']['pageProps']['seasonLiftTickets'][x]
            else:
                prices[x + "_season"] = np.nan

    return prices

if __name__ == '__main__':
    """
    Extract data for each ski resort.
    """
    resortData = dict()
    
    for url in url_df['url']:
        resortUrl = url
        stat = get_basic_resort_statistics(resortUrl)
        prices = get_resort_prices(resortUrl)
        time.sleep(1)

        resortData[url] = {**stat, **prices, "Url": resortUrl}

    df = pd.DataFrame.from_dict(resortData, orient='index')
    #df.to_csv('data/OnTheSnow_Srape_2_200523.csv', index=False)
    df.to_csv('data/OnTheSnow_Scrape_2_820523.csv', index=False)

In [295]:
df

,ski_resort,address,city,state,country,summit,drop,base,gondolas_and_trams,fast_eight,...,teenagerPrice_season,adultPrice_season,seniorPrice_season,Url,gondolas_lifts_note,beginner_runs,intermediate_runs,advanced_runs,expert_runs,night_skiing
https://www.onthesnow.com/california/squaw-valley-usa/ski-resort.html,Palisades Tahoe,PO Box 2007,96146 Olympic Valley,California,United States,9050,2850,6200,3.0,6.0,...,879.00,"1,179.00",None,https://www.onthesnow.com/california/squaw-val...,NaN,NaN,NaN,NaN,NaN,NaN
https://www.onthesnow.com/california/mammoth-mountain-ski-area/ski-resort.html,Mammoth,P.O. Box 24,93546 Mammoth Lakes,California,United States,11053,3100,7953,3.0,9.0,...,879.00,"1,179.00",None,https://www.onthesnow.com/california/mammoth-m...,Check gondola information.,15%,48%,24%,13%,NaN
https://www.onthesnow.com/california/donner-ski-ranch/ski-resort.html,Donner Ski Ranch,P.O. Box 66,95724 Norden,California,United States,8012,750,7031,NaN,NaN,...,449.00,499.00,449.00,https://www.onthesnow.com/california/donner-sk...,NaN,31%,38%,21%,10%,NaN
https://www.onthesnow.com/california/sugar-bowl-resort/ski-resort.html,Sugar Bowl,P.O. Box 5,95724 Norden,California,United States,8383,1500,6883,1.0,5.0,...,889.00,"1,119.00",889.00,https://www.onthesnow.com/california/sugar-bow...,NaN,15%,45%,28%,12%,NaN
https://www.onthesnow.com/california/kirkwood/ski-resort.html,Kirkwood,PO Box 1,95646 Kirkwood,California,United States,9800,2000,7800,NaN,2.0,...,429.00,545.00,437.00,https://www.onthesnow.com/california/kirkwood/...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://www.onthesnow.com/new-york/oak-mountain/ski-resort.html,Oak Mountain,141 Novosel Way,12164 Speculator,NY,United States,2400,650,1750,NaN,NaN,...,315.00,415.00,315.00,https://www.onthesnow.com/new-york/oak-mountai...,NaN,45%,27%,18%,9%,12 ac
https://www.onthesnow.com/pennsylvania/mount-pleasant-of-edinboro/ski-resort.html,Mount Pleasant,23301 Plank Rd,16403 Venango,Pa 16440,United States,1540,340,1200,NaN,NaN,...,None,350.00,None,https://www.onthesnow.com/pennsylvania/mount-p...,NaN,22%,56%,22%,NaN,35 ac
https://www.onthesnow.com/new-york/hunt-hollow-ski-club/ski-resort.html,Hunt Hollow,7532 County Road 36,14512 Naples,New York,United States,2030,825,1000,NaN,NaN,...,None,None,None,https://www.onthesnow.com/new-york/hunt-hollow...,NaN,32%,21%,37%,11%,400 ac
https://www.onthesnow.com/connecticut/powder-ridge-park/ski-resort.html,Powder Ridge,99 Powder Hill Road,06455 Middlefield,Connecticut,United States,720,550,170,NaN,NaN,...,520.00,570.00,520.00,https://www.onthesnow.com/connecticut/powder-r...,NaN,45%,40%,15%,NaN,40 ac


In [296]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 331 entries, https://www.onthesnow.com/california/squaw-valley-usa/ski-resort.html to https://www.onthesnow.com/pennsylvania/shawnee-mountain-ski-area/ski-resort.html
Data columns (total 61 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ski_resort            331 non-null    object 
 1   address               331 non-null    object 
 2   city                  331 non-null    object 
 3   state                 331 non-null    object 
 4   country               331 non-null    object 
 5   summit                331 non-null    int64  
 6   drop                  331 non-null    int64  
 7   base                  331 non-null    int64  
 8   gondolas_and_trams    40 non-null     float64
 9   fast_eight            110 non-null    float64
 10  high_speed_sixes      41 non-null     float64
 11  quad_chairs           163 non-null    float64
 12  triple_chairs         224 non-null  